<a href="https://colab.research.google.com/github/IamMarking/Tensorflow/blob/master/7_2%E9%80%92%E5%BD%92%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9CRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 梯度消失
- 

[LSTM 学习笔记1](https://zhuanlan.zhihu.com/p/32085405)

[LSTM 学习笔记2](https://www.jianshu.com/p/9dc9f41f0b29)

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from tensorflow.contrib import rnn  # 导入 rnn 这个类

In [0]:
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

# 输入图片是 28*28
n_inputs = 28 #输入一行，一行有28个数据
max_time = 28 #一共28行
lstm_size = 100 # 隐层单元
n_classes = 10 # 10个分类

batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples // batch_size #计算一共有多少个批次

#这里的none表示第一个维度可以是任意的长度
x = tf.placeholder(tf.float32,[None,784])
#正确的标签
y = tf.placeholder(tf.float32,[None,10])

#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))

#初始化偏置值
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))

In [0]:
#定义RNN网络
def RNN(X,weights,biases):  # 传入的参数有 3 个维度 批次大小 多少行 一行有多少数据
    # inputs=[batch_size, max_time, n_inputs]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])  # 改变形状
    #定义LSTM基本CELL
#     lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size)
    lstm_cell = rnn.BasicLSTMCell(lstm_size) 
  
    # final_state[0]是 cell state
    # final_state[1]是 hidden_state
    # final_state[state,batch_size,cell.state.size]  # 隐藏单元的个数
    
    # outputs time_major=False  [batch_size,max_time,ell.state.size]
    # True [max_time,batch_size,cell.state.size]
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases)
    
    return results

In [3]:
inputs = tf.Variable(tf.random_normal([100,28,28]))
weights = tf.Variable(tf.truncated_normal([100, 10], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[10]))

lstm_cell = rnn.BasicLSTMCell(num_units=100,state_is_tuple=True)

print('lstm_cell',lstm_cell)

outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
print('outputs', outputs)
print('final_state', final_state)

results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases)
print(results)

init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  
  print(sess.run(lstm_cell))
  print(sess.run(outputs))
  print(sess.run(results))

lstm_cell <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7f24c6cca080>


ValueError: ignored

In [0]:
#计算RNN的返回结果
prediction= RNN(x, weights, biases)  

#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))

#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#把correct_prediction变为float32类型

#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):  # 训练多少轮
        for batch in range(n_batch):  # 一轮要训练多少个 batch
          
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)  # 不断地去拿下 100 个数据
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter " + str(epoch) + ", Testing Accuracy= " + str(acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Iter 0, Testing Accuracy= 0.7398
Iter 1, Testing Accuracy= 0.8614
Iter 2, Testing Accuracy= 0.8962
Iter 3, Testing Accuracy= 0.92
Iter 4, Testing Accuracy= 0.9238
Iter 5, Testing Accuracy= 0.9241
